In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# 0) Cross-Encoder modeli yükle
model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSequenceClassification.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 1) Ham veriyi yükle
raw_train = pd.read_csv("wikiqa_train.csv").reset_index(drop=True)
raw_test  = pd.read_csv("wikiqa_test.csv").reset_index(drop=True)

# 2) CE skorlarını hesapla
def compute_ce_scores(df):
    scores = []
    batch_size = 32
    for i in tqdm(range(0, len(df), batch_size), desc="CE scoring"):
        batch = df.iloc[i : i + batch_size]
        inputs = tokenizer(
            batch["question"].tolist(),
            batch["answer"].tolist(),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            scores.extend(logits.squeeze(-1).cpu().tolist())
    return scores

# 3) CE skoru ekle
feat_train = pd.read_csv("final_feature_set_train.csv").reset_index(drop=True)
feat_test  = pd.read_csv("final_feature_set_test.csv").reset_index(drop=True)

feat_train["ce_score"] = compute_ce_scores(raw_train)
feat_test["ce_score"]  = compute_ce_scores(raw_test)

# Etiketleri ayır
X_train_meta = feat_train.drop(columns=["question_id", "label"])
y_train_meta = feat_train["label"]
X_test_meta = feat_test.drop(columns=["question_id", "label", "proba_meta", "pred_meta"], errors='ignore')
y_test_meta = feat_test["label"].astype(int)

# ----------------------------------------
# 2) scale_pos_weight Hesapla
# ----------------------------------------
neg = (y_train_meta == 0).sum()
pos = (y_train_meta == 1).sum()
scale_pos = neg / pos
print(f"🔢 scale_pos_weight = {scale_pos:.2f}")

# ----------------------------------------
# 3) XGBoost Modelini Eğit
# ----------------------------------------
xgb_clf = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
xgb_clf.fit(X_train_meta, y_train_meta)

# ----------------------------------------
# 4) Test Verisi Üzerinde proba_meta Hesapla
# ----------------------------------------
proba_meta = xgb_clf.predict_proba(X_test_meta)[:, 1]
feat_test["proba_meta"] = proba_meta

# ----------------------------------------
# 5) Threshold Taramasıyla En İyi Eşiği Bul
# ----------------------------------------
best_threshold = 0.0
best_f1 = 0.0
thresholds = np.arange(0.1, 0.91, 0.05)
results = []

best_idxs = feat_test.groupby("question_id")["proba_meta"].idxmax()

for t in thresholds:
    pred_meta = np.zeros(len(feat_test), dtype=int)
    best_probas = feat_test.loc[best_idxs, "proba_meta"]
    valid_best = best_probas[best_probas > t].index
    pred_meta[valid_best] = 1

    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test_meta, pred_meta, average="binary", zero_division=0
    )

    results.append((t, precision, recall, f1))

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Sonuçları yazdır
print(f"\n🔍 Best threshold: {best_threshold:.2f} with F1-score: {best_f1:.3f}\n")
print("Threshold | Precision | Recall | F1")
for r in results:
    print(f"{r[0]:.2f}       | {r[1]:.3f}     | {r[2]:.3f}  | {r[3]:.3f}")

# ----------------------------------------
# 6) En İyi Threshold ile Nihai Tahmin
# ----------------------------------------
feat_test["pred_meta"] = 0
best_probas = feat_test.loc[best_idxs, "proba_meta"]
valid_best = best_probas[best_probas > best_threshold].index
feat_test.loc[valid_best, "pred_meta"] = 1

# ----------------------------------------
# 7) Sonuçları Yazdır
# ----------------------------------------
print("\n📊 Final Report (Threshold Applied):")
print(classification_report(y_test_meta, feat_test["pred_meta"], digits=3))
print("🔎 Confusion Matrix:")
print(confusion_matrix(y_test_meta, feat_test["pred_meta"]))

print(X_train_meta.columns.tolist())